In [238]:
import time
import requests
import pandas as pd
import json
import requests

In [239]:
requests.packages.urllib3.disable_warnings()

In [241]:
def use_VT_api_for_ip_adress(ip_adress: str) -> requests.models.Response:
    url = "https://www.virustotal.com/api/v3/ip_addresses/{}".format(ip_adress)
    headers = {
        "accept": "application/json",
        "x-apikey": "cda6ca0fa003aaf96623ee0ffe86379342f859a476a7153325215dac42ea0220" #fake
    }

    return requests.get(url, headers=headers, verify=False)

In [242]:
def create_report_row_ip(vuln_host_ip: str, suspect_ip: str) -> pd.DataFrame:
    response = use_VT_api_for_ip_adress(suspect_ip)
    json_report = json.loads(response.text)
    try:    
        json_report = json_report['data']['attributes']
    except:
        print(json_report)
        print(vuln_host_ip, suspect_ip)
    json_final_report = {
        suspect_ip: {
        "vuln_host": vuln_host_ip,
        "score (VirusTotal)": "{}/{}".format(json_report['last_analysis_stats']['malicious'], 
                                               sum(json_report['last_analysis_stats'].values()))
        }
    }
    tags = ["malware", "malicious", "phishing"]
    for tag in tags:
        if tag in response.text:
            json_final_report[suspect_ip][tag] = response.text.count('"result": "{}"'.format(tag))
    return pd.DataFrame.from_dict(json_final_report, orient='index')


<h1>ZAIMPORTOWANIE JSONA ALERTÓW</h1>


In [307]:
alerts = json.load(open('alert.json'))

Scalanie jsonow

In [295]:
alerts2 = json.load(open('alert.json'))
for hit in alerts2['hits']['hits']:
    alerts['hits']['hits'].append(hit)

In [308]:
len(alerts['hits']['hits'])

16

In [ ]:
init_table = [ 
    (hit['fields']['source.ip'][0], hit['fields']['destination.ip'][0]) for hit in alerts['hits']['hits']
]
init_table = list(set(init_table))
init_table

In [310]:
len(init_table)

5

In [311]:
##################################### RULE INFO #####################################
print("Description:\n", alerts['hits']['hits'][0]['_source']['signal']['rule']['description'])
print("\nFalse positives: ")
for fp in alerts['hits']['hits'][0]['_source']['signal']['rule']['false_positives']:
    print(fp)

Description:
 Generates a detection alert for each external alert written to the configured indices. Enabling this rule allows you to immediately begin investigating external alerts in the app.

False positives: 


In [312]:
##################################### RULE QUERY #####################################
' '.join(alerts['hits']['hits'][0]['fields']['signal.rule.query'][0].split())

'event.kind:alert and not event.module:(endgame or endpoint)'


<h1>RAPORT VIRUSTOTAL</h1>


In [313]:
final_report = pandas.concat([create_report_row_ip(ips[1], ips[0]) for ips in init_table], axis=0)

In [314]:
final_report_cp = final_report

In [315]:
final_report_cp.sort_values(['malware', 'malicious', 'vuln_host'], inplace=True, ascending=False)

In [316]:
final_report_cp = final_report_cp[(final_report_cp['score (VirusTotal)'] != '0/88') & (final_report_cp['score (VirusTotal)'] != '0/87')]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 3):
    print(final_report_cp)